In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticOutputParser
from typing import List
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
from langchain.output_parsers import OutputFixingParser
from operator import itemgetter
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI  # example LLM
from langchain.output_parsers import StructuredOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnablePassthrough, chain, RunnableConfig
import pandas as pd
import re
import os
from langchain_openai import OpenAI
from dotenv import load_dotenv
load_dotenv()




True

In [2]:

llm = OpenAI(name="gpt-4o")


In [3]:
llm.invoke("Hello, world!")

'\n\nThis is a test\n\nNice to meet you, test. I am a computer program created to perform tasks and assist humans in various ways. How can I help you today?'

In [4]:
llm = OpenAI(name="o3")

In [5]:
llm.invoke("Hello, world!")

"\n\nHello, world! It's nice to meet you. My name is OpenAI. I am an artificial intelligence programmed to assist and communicate with humans. How may I be of service to you?"

In [6]:
prompt = PromptTemplate.from_template("""
    How to say {input} in {target_language} language
""")           

chain = prompt | llm | StrOutputParser()


chain.invoke({"input": "Danke", "target_language": "Russian"})

'\nСпасибо (Spasibo)'

In [7]:
def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

folder_path = "documents"
documents = load_documents(folder_path)

len(documents)


42

In [8]:
documents

[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 14.0 (Windows)', 'creationdate': '2022-10-25T14:23:09+03:00', 'moddate': '2022-10-25T14:23:12+03:00', 'trapped': '/False', 'source': 'documents/_upload_iblock_723_v18v3w60o611z0grp7svpcq0ul5nv76p.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content='ИНСТРУКЦИЯ\nТиволи 4 см «римская кладка»\nпо укладке тротуарной плитки'),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 14.0 (Windows)', 'creationdate': '2022-10-25T14:23:09+03:00', 'moddate': '2022-10-25T14:23:12+03:00', 'trapped': '/False', 'source': 'documents/_upload_iblock_723_v18v3w60o611z0grp7svpcq0ul5nv76p.pdf', 'total_pages': 5, 'page': 1, 'page_label': '2'}, page_content='1. Для создания рисунка “римской кладки” используйте одно из шести сочетаний \nэлементов тротуарных плит Тиволи производства компании White Hills.\nВариант 1\nВариант 2'),
 Document(metadata={'producer': 'Adobe PDF Library 15.

In [9]:
[doc.metadata for doc in documents]

[{'producer': 'Adobe PDF Library 15.0',
  'creator': 'Adobe InDesign 14.0 (Windows)',
  'creationdate': '2022-10-25T14:23:09+03:00',
  'moddate': '2022-10-25T14:23:12+03:00',
  'trapped': '/False',
  'source': 'documents/_upload_iblock_723_v18v3w60o611z0grp7svpcq0ul5nv76p.pdf',
  'total_pages': 5,
  'page': 0,
  'page_label': '1'},
 {'producer': 'Adobe PDF Library 15.0',
  'creator': 'Adobe InDesign 14.0 (Windows)',
  'creationdate': '2022-10-25T14:23:09+03:00',
  'moddate': '2022-10-25T14:23:12+03:00',
  'trapped': '/False',
  'source': 'documents/_upload_iblock_723_v18v3w60o611z0grp7svpcq0ul5nv76p.pdf',
  'total_pages': 5,
  'page': 1,
  'page_label': '2'},
 {'producer': 'Adobe PDF Library 15.0',
  'creator': 'Adobe InDesign 14.0 (Windows)',
  'creationdate': '2022-10-25T14:23:09+03:00',
  'moddate': '2022-10-25T14:23:12+03:00',
  'trapped': '/False',
  'source': 'documents/_upload_iblock_723_v18v3w60o611z0grp7svpcq0ul5nv76p.pdf',
  'total_pages': 5,
  'page': 2,
  'page_label': '3'}

In [10]:
os.listdir("documents")

['_upload_iblock_723_v18v3w60o611z0grp7svpcq0ul5nv76p.pdf',
 '_upload_iblock_606_nlf798sqxiddt0jddmfg8uy6r1qec2tp.pdf',
 '_upload_iblock_023_0235e4bc383926f54e57888839843405.pdf',
 '_upload_iblock_e5b_e5bc5b097ef730e4719c4a20e4591d6f.pdf',
 '_upload_iblock_1f9_1f9e55bb3fc71c70f3e02d3d144682c0.pdf',
 '_upload_iblock_d18_d18ed70db7b40893f7bd2a69922658e6.pdf',
 '_upload_iblock_ce6_ce62808ad9b1e4736343d7668b99fd5b.pdf',
 '_upload_iblock_d7e_d7e210e9d0593b74f8f96866b28a4ab9.pdf',
 '_upload_iblock_f59_f1aphtbdgbjobtc383px49qnexz8erep.pdf',
 '_upload_iblock_6c0_56uylwl41jxefdrlk9i8d1lmt6pbwrr3.pdf',
 '_upload_iblock_955_955d9f1eee6a824fdd9312beed408a12.pdf',
 '_upload_iblock_c1b_c1b39be5c349c4ffd1459260c0ae0b8c.pdf']

In [11]:
# text classification
from langchain_text_splitters import RecursiveCharacterTextSplitter

documents = load_documents("documents")
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(documents)
len(chunks)

56

In [12]:

auto_label_prompt = PromptTemplate.from_template("""
Come up with a short, general topic name for the following text.

Text:
{chunk}

Return only the topic name.
""")
auto_label_chain = auto_label_prompt | llm | StrOutputParser()

results = []

for chunk in chunks[:10]:  
    topic = auto_label_chain.invoke({
        "chunk": chunk.page_content
    })
    results.append({
        "text": chunk.page_content[:50],
        "topic": topic
    })

import pandas as pd
df = pd.DataFrame(results)
df.tail()


,text,topic
5,продукции. Тротуары должны быть выполнены в со...,"\n""Тротуары и правила заключения сделок"""
6,ГИДРОФОБИЗАТОР\nВодоотталкивающий кремнийорган...,\nWater-repellent Coatings
7,паропроницаемости. Глубина проникновения раств...,"\n""Waterproof Coatings"""
8,Способ применения\nПодготовка поверхности\nПов...,"\n""Surface Preparation and Application"""
9,"элемент, проверить соответствие цвета обработа...",\nSurface Coating and Application


In [13]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
model_kwargs = {'trust_remote_code': True
                # ,'device': 'cuda'
                }
#model_name='models/Linq-Embed-Mistral'
#model_name='models/multilingual-e5-large-instruct'
model_name='intfloat/multilingual-e5-large-instruct'
embedding_model = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
document_embeddings = embedding_model.embed_documents([split.page_content for split in chunks])
print(f"Created embeddings for {len(document_embeddings)} document chunks.")



Created embeddings for 56 document chunks.


In [14]:
document_embeddings

[[0.015244313515722752,
  0.025815078988671303,
  -0.00017016337369568646,
  -0.024249015375971794,
  0.04030679166316986,
  -0.01858506165444851,
  -0.039680853486061096,
  0.09114708006381989,
  0.04458654671907425,
  -0.008510253392159939,
  0.029982132837176323,
  0.04569115862250328,
  -0.015373732894659042,
  -0.009119697846472263,
  -0.013342056423425674,
  -0.026050636544823647,
  -0.09502255171537399,
  0.027548952028155327,
  -0.01617269404232502,
  -0.031854692846536636,
  0.02757217548787594,
  0.0018295602640137076,
  -0.0540582537651062,
  -0.03547796234488487,
  -0.009583893232047558,
  -0.015363489277660847,
  0.0014228647341951728,
  -0.042256034910678864,
  -0.010475803166627884,
  -0.036158427596092224,
  -0.0035822810605168343,
  0.028649240732192993,
  -0.004906746558845043,
  -0.027645180001854897,
  -0.023675506934523582,
  0.020624952390789986,
  0.03785974159836769,
  0.03606755658984184,
  -0.054230742156505585,
  0.04556960612535477,
  0.0024356369394809008,


In [15]:
from langchain_chroma import Chroma

collection_name = "my_collection"
vectorstore = Chroma.from_documents(
    collection_name=collection_name,
    documents=chunks,
    embedding=embedding_model,
    persist_directory="./chroma_db"
)
print("Vector store created and persisted to './chroma_db'")

Vector store created and persisted to './chroma_db'


In [20]:
query = "установка бордюрного камня"
search_results = vectorstore.similarity_search(query, k=2)
print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()



Top 2 most relevant chunks for the query: 'установка бордюрного камня'

Result 1:
Source: documents/_upload_iblock_ce6_ce62808ad9b1e4736343d7668b99fd5b.pdf
Content: 4. Установка бордюрного камня.  Бордюрные камни устанавливаются на свежезалитое бетонное 
основание. После его установки устраивается бетонная обойма в опалубке на высоту 10 см с внешней стороны 
и на 5 см с внутренней стороны. Бордюр должен быть установлен не менее чем за сутки до начала работ по 
устройству бетонного основания , чтобы бетон обоймы набрал достаточную прочность. 
 
5. Устройство бетонного основания. Основание состоит из нескольких слоев. Засыпьте слой щебня 
фракции 5 -20 мм или 20 -40 мм. Толщина слоя после трамбовки должна быть – 10 см. Утрамбуйте щебень 
виброплитой или ручным трамбов щиком. Установите дорожную сетку 100Х100Х4 по всей площади участка. 
При необходимости стыкуйте сетку внахлест (не менее 20 см). Высота сетки над слоем щебня должна быть 7-8 
см, для того, чтобы после заливки бетонного осн